In [30]:
# Import parquet file with pySpark
from pyspark.sql import SparkSession

# Create a spark session
spark = spark = SparkSession.builder \
                            .appName("ML-part") \
                            .master("local[*]") \
                            .config("spark.driver.memory", "16g") \
                            .config("spark.executor.memory", "16g") \
                            .getOrCreate()
# Read parquet file
df = spark.read.parquet(r"./simplified_train_data.parquet")
df_test = spark.read.parquet(r"./simplified_test_data.parquet")

# Show the data
df.printSchema()


root
 |-- attack_index: double (nullable = true)
 |-- scaled_ttl_features: vector (nullable = true)
 |-- scaled_tcp_flags_features: vector (nullable = true)
 |-- scaled_flow_features: vector (nullable = true)
 |-- scaled_duration_features: vector (nullable = true)
 |-- scaled_pkt_size_features: vector (nullable = true)
 |-- scaled_tcp_win_features: vector (nullable = true)
 |-- scaled_l4_dst_port_features: vector (nullable = true)



In [31]:
from pyspark.ml.feature import VectorAssembler

# Preparar os dados com VectorAssembler
feature_columns = df.columns
feature_columns.remove("attack_index")
print(feature_columns)
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_assembled = assembler.transform(df)

['scaled_ttl_features', 'scaled_tcp_flags_features', 'scaled_flow_features', 'scaled_duration_features', 'scaled_pkt_size_features', 'scaled_tcp_win_features', 'scaled_l4_dst_port_features']


In [32]:
from pyspark.ml.classification import RandomForestClassifier, NaiveBayes, LogisticRegression, MultilayerPerceptronClassifier

# Definir o Random Forest
rf = RandomForestClassifier(labelCol="attack_index", featuresCol="features", numTrees=10)

# Definir o Naive Bayes
#nb = NaiveBayes(labelCol="attack_index", featuresCol="features")

# Definir o Logistic Regression
lr = LogisticRegression(labelCol="attack_index", featuresCol="features")

# Configuração do Multilayer Perceptron
# Obter o número de características
def get_feature_count(df, feature_col="features"):
# Extrai os metadados da coluna de características e calcula a soma dos tamanhos dos atributos
    attributes = df.schema[feature_col].metadata["ml_attr"]["attrs"]
    feature_count = sum(len(attrs) for attrs in attributes.values())
    return feature_count

print(get_feature_count(df_assembled, "features"))

input_layers = get_feature_count(df_assembled, "features")

output_layers = df_assembled.select("attack_index").distinct().count()
print(output_layers)
mlp = MultilayerPerceptronClassifier(
    labelCol="attack_index", 
    featuresCol="features", 
    layers=[7, 9, 13, 11, 10],
    maxIter=100
)

13
10


In [33]:
# Random Forest
model_rf = rf.fit(df_assembled)
#model_nb = nb.fit(df_assembled)
# logistic regression
model_lr = lr.fit(df_assembled)
# Multilayer Perceptron
model_mlp = mlp.fit(df_assembled)

In [34]:
from unittest import result


result_rf = model_rf.transform(assembler.transform(df_test))
result_lr = model_lr.transform(assembler.transform(df_test))
result_mlp = model_mlp.transform(assembler.transform(df_test))

In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Avaliador
evaluator = MulticlassClassificationEvaluator(labelCol="attack_index", predictionCol="prediction", metricName="accuracy")

# Avaliar Random Forest
accuracy_rf = evaluator.evaluate(result_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")

# Avaliar Naive Bayes
#accuracy_nb = evaluator.evaluate(model_nb.transform(df_assembled))
#print(f"Naive Bayes Accuracy: {accuracy_nb}")





Random Forest Accuracy: 0.835


In [36]:
# Avaliar Logistic Regression
accuracy_lr = evaluator.evaluate(result_lr)
print(f"Logistic Regression Accuracy: {accuracy_lr}")

Logistic Regression Accuracy: 0.665


In [37]:
# Avaliar Multilayer Perceptron

accuracy_mlp = evaluator.evaluate(result_mlp)
print(f"Multilayer Perceptron Accuracy: {accuracy_mlp}")

Py4JJavaError: An error occurred while calling o2444.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1248.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1248.0 (TID 1332) (c16636a8749b executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`ProbabilisticClassificationModel$$Lambda$4578/0x00007f5a8d2158d0`: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasN